## Problem

I want to get the column and row indices of a grid that corresponds to latitude and longitude. 

## Solution

In Python you can use the `pyproj` package to transform latitude and longitude (geographic coordinates) to x and y (projected coordinates) in the EASE Grid CRS, and then use the `affine` package to get the column and row indices (`(i,j)` image coordinates).

In [5]:
from pyproj import Transformer
from affine import Affine
import numpy as np

# Define the coordinate transform from WGS84 (lat, lon) 
# to CRS of your grid using EPSG codes.
# WGS84 is EPSG:4326
# NSIDC Northern Hemisphere EASE Grid is EPSG:3408
transformer = Transformer.from_crs(4326, 3408)

# Define the affine transformation matrix
grid_cell_width = 50057.094722222224
grid_cell_height = -50057.094722222224
upper_left_x = -9036842.76
upper_left_y = 9036842.76
geotransform = Affine(grid_cell_width, 0.0, upper_left_x,
                      0.0, grid_cell_height, upper_left_y)

x, y = transformer.transform(-45., 84.)  # latitude and longitude
i, j = ~geotransform * (x, y)  # ~geotransform is the inverse operation

print(i,j)

414.4236938226952 205.11385157648664


`i` and `j` are in image coordinates with the origin as the upper-left corner of the upper-left grid-cell.  To get the column and row indices of the cell you can use `floor`.

In [9]:
col, row = np.floor(i).astype(int), np.floor(j).astype(int)
print(col, row)

414 205


## Discussion

Many grids, whether they are images or gridded data are in _projected coordinate systems_ so that they can be displayed on a 2D screen.  Converting from geographic coordinates (_latitude_, _longitude_) to image coordinates (_column_, _row_) is usually a two step process.  In the first step, the geographic coordinates are transformed from a Geographic Coordinate System with coordinates _latitude_ and _longitude_ to the _projected coordinate system_ of the image or grid.  This is a _Cartesian_ coordinate system with coordinates _x_ and _y_, usually in meters.  In the second step, the _projected coordinates_ are converted into _image coordinates_.  This is another _Cartesian_ coordinate system with coordinates _column_ and _row_ or _i_ and _j_ with units grid-cells.  Image coordinate systems often have the origin in the upper-left corner of the upper-left grid cell.

In Python the first step is easily acheived using the `pyproj` package.  If the Coordinate Reference System (CRS) of the grid is registered in the EPSG database, defining the transformation is easily done with the EPSG codes for the CRS for the _latitude_ and _longitude_ coordinates and the CRS of the destination grid.  Usually, _latitude_ and _longitude_ are in the Ensemble WGS84 CRS, which is EPSG:4326.  For NSIDC gridded data, the CRS is some flavour of EASE Grid or Polar Stereographic.  See the help pages for [EASE Grid](https://nsidc.org/data/user-resources/help-center/guide-ease-grids) and [Polar Stereographic](https://nsidc.org/data/user-resources/help-center/guide-nsidcs-polar-stereographic-projection) for more information. 

:::{note}
Both EASE Grid and Polar Stereographic CRS come in two main flavours _Original_ and WGS84.  For EASE Grids, these two flavours are EASE-Grid and EASE-Grid v2.0.
:::

TODO: Describe projected to image coordinate conversion

Add image showing the relationship between the three coordinate systems